#Transformation logic

In [0]:
query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.customer_id) AS customer_key,
    ci.customer_id,
    ci.customer_number,
    ci.first_name,
    ci.last_name,
    la.country,
    ci.marital_status,
    CASE
        WHEN ci.gender <> 'n/a' THEN ci.gender
        ELSE COALESCE(ca.gender, 'n/a')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.create_date AS create_date
FROM silver.crm_cust ci
LEFT JOIN silver.erp_customers ca
    ON ci.customer_number = ca.customer_number
LEFT JOIN silver.epr_loc la
    ON ci.customer_number = la.customer_number
"""
df = spark.sql(query)

##check table


In [0]:
df.limit(10).display()

#Writing gold table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_customer")

#check gold table

In [0]:
%sql
select*from workspace.gold.dim_customer